# Import libraries

In [ ]:
import vortexasdk as v
import pandas as pd
from datetime import datetime
import numpy as np
import time
import dateutil.relativedelta
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.tsa.stattools import grangercausalitytests

# Search Geography & Product IDs

In [ ]:
# search for geography ids (remove hashtags to search)

# full_length_df = v.Geographies().search(term=["US Gulf Coast"]).to_df()
# print(full_length_df.to_string(index=False))

meg='0427e0f9d52b38c1a98b68c59b8fd80cb1c508e44882e96611e48ef5b140d927'
padd3='4e79eb8e84d26f5c3c0006283bc1aa52c170b58d667be9848e136afea91a57e9'
nwe='c5460c5a4ece7b64ffc0cc280aeade60d364423e8e062ef4a11494352fe6fdbb'
gom='37c8c4eeb730d1cd41f90ca6bf95c923222b0734b1b0336a475acce821f87ebd'

# search for product ids (remove hashtags below to search)

# product_search = v.Products().search(term=['naphtha']).to_df()
# print (product_search.to_string(index=False))

cpp='b68cbb746f8b9098c50e2ba36bcad83001a53bd362e9031fb49085d02c36659c'
lpg='364ccbb996c944055b479810a8e74863267885dc1b01407cb0f00ab26dafe1e1'
naphtha='3e4db72ef7027de928ce55703a213a546fd86d2debe6f2e9c85f3a5f9d53e8dd'
diesel_go='deda35eb9ca56b54e74f0ff370423f9a8c61cf6a3796fcb18eaeeb32a8c290bb'
crude_co='54af755a090118dcf9b0724c9a4e9f14745c26165385ffa7f1445bc768f06f11'

# Functions

In [ ]:
# function to create a moving average
def moving_average(data, period, option):
    
    if option=='multiple':

        # calculate moving avg
        moving_avg = pd.DataFrame(data.iloc[:, 1:].rolling(window=period, min_periods=1).mean())

        # add moving average
        moving_avg_df=pd.concat([data.iloc[0:, 0:1], moving_avg], axis=1)

        moving_avg_df.columns=list(data.columns)
        
    elif option=='single':
        
        # calculate moving avg
        moving_avg = pd.DataFrame(data['value'].rolling(window=period, min_periods=1).mean())
        moving_avg.columns=[f'{period}-day moving_avg']

        # get all columns
        data_cols=list(data.columns)

        # get all columns except vlaue
        date_cols=[x for x in data_cols if x !='value']

        # add moving average
        moving_avg_df=pd.concat([data[date_cols], moving_avg], axis=1)

        moving_avg_df.rename(columns={f'{period}-day moving_avg':'value'}, inplace=True)
        

    return moving_avg_df

# Freight rates function
def freight_rates(start_y, start_m, start_d, rates, unit, freq, plot, title):
    
    # Set current date
    today=datetime.today()
    
    # empty data frame to loop through and store
    final = pd.DataFrame()
    
    # Obtain dates object
    dates=v.FreightPricingTimeseries().search(
        time_min=datetime(start_y, start_m, start_d),
        time_max=today,
        routes=rates[0],
        breakdown_property=unit,
        breakdown_frequency=freq).to_df()
    
    # take just dates column
    dates = pd.concat([dates["key"]], axis = 1)
    final = dates
    final.columns = ['Date']
    
    # Loop through route codes to obtain each route's freight rates
    for i in range(len(rates)):
        df=v.FreightPricingTimeseries().search(
            time_min=datetime(start_y, start_m, start_d),
            time_max=today,
            routes=rates[i],
            breakdown_property=unit,
            breakdown_frequency=freq).to_df()

        df2 = df["value"]
        final = pd.concat([final, df2], axis = 1)

    names = ['Date'] + rates
    final.columns = names
    final['Date']=pd.to_datetime(final['Date'])
    
        
    # Replace blanks with pandas NA values
    final.replace('', pd.NA, inplace=True)
    
    # Remove NAs
    final.dropna(inplace=True)

    # If desired, plot rates
    if plot:
        
        # Plot rates
        fig = px.line(
            final,
            x="Date", 
            y=list(final.columns)[1:],
            title=title,
            labels={
                "Date":"Date",
            },
            )
        fig.update_layout(xaxis_rangeslider_visible = True)
        fig.show()
    
    # Reformat dates for export to excel
    final['Date']=final['Date'].dt.strftime("%d-%m-%Y")

    
    return final


# Vessel avaialbility function
def vessel_availability(start_y, start_m, start_d, region, port, prod, prod_excl, vessel_class, vessel_class_excl, heading_to, heading_to_excl, status, laycan_min, laycan_max, ma_period, plot, title):

    today=datetime.today()
    
    # Pull avalability time series data
    ts_result = v.VesselAvailabilityTimeseries().search(
        filter_time_min=datetime(start_y, start_m, start_d),
        filter_time_max=today,
        filter_region=region,
        filter_port=port,
        filter_products=prod,
        exclude_products=prod_excl,
        filter_vessel_classes=vessel_class,
        exclude_vessel_classes=vessel_class_excl,
        filter_destination=heading_to,
        exclude_destination=heading_to_excl,
        filter_days_to_arrival=[{"min": laycan_min, "max": laycan_max}], # specify laycan window
        filter_vessel_scrubbers="disabled",
        filter_vessel_status=status,
        use_reference_port=True
        ).to_df()
    
    ts_result=pd.concat([ts_result["key"], ts_result["count"]], axis=1)
    ts_result.columns=["Date", 'Availability']
    
    if ma_period != None:
        ts_result=moving_average(data=ts_result, period=ma_period, option='multiple')
        ts_result.columns=["Date", f'Availability ({ma_period}-day MA)']

    
    if plot: # plot data if desired

        fig = px.line(
            ts_result, # data to plot
            title=title, # title set as input
            x="Date",
            y=list(ts_result.columns)[1:],
            labels={
                "value":'No. of vessels' # unit label
            },
            )
        fig.update_layout(xaxis_rangeslider_visible = True)
        fig.show()
    
    ts_result['Date']=ts_result['Date'].dt.strftime('%d-%m-%Y')
    
    return ts_result

def fr_va_combination(start_y, start_m, start_d, region, port, prod, prod_excl, vessel_class, vessel_class_excl, heading_to, heading_to_excl, status, laycan_min, laycan_max, va_ma_period, rates, unit, plot, title):
    
    
    frs_df=freight_rates(start_y=start_y, start_m=start_m, start_d=start_d,  
                         rates=rates, unit=unit, freq='day', 
                         plot=False, title='freight rates')


    va_df=vessel_availability(start_y=start_y, start_m=start_m, start_d=start_d, 
                              region=region, port=port, 
                              prod=prod, prod_excl=prod_excl, 
                              vessel_class=vessel_class, vessel_class_excl=vessel_class_excl, 
                              heading_to=heading_to, heading_to_excl=heading_to_excl, 
                              status=status, laycan_min=laycan_min, laycan_max=laycan_max, 
                              ma_period=va_ma_period,
                              plot=False, title='availability')
    
    combined_df=pd.merge(frs_df, va_df, on='Date', how='left')
    
    combined_df['Date']=pd.to_datetime(combined_df['Date'], format="%d-%m-%Y", errors='coerce')
    
    # Convert all integer columns to floats
    combined_df[list(combined_df.columns)[1]]=combined_df[list(combined_df.columns)[1]].astype(float)
    combined_df[list(combined_df.columns)[2]]=combined_df[list(combined_df.columns)[2]].astype(float)
    
    if unit=='cost':
        fr_unit='$/ton'
        
    elif unit=='tce':
        fr_unit='$/day'
        
    elif unit=='route':
        fr_unit='WS'

    if plot:
        
        fig = go.Figure()

        # Add first line (LHS y-axis)
        fig.add_trace(
            go.Scatter(
                x=combined_df['Date'],
                y=combined_df[list(combined_df.columns)[1]],
                name=list(combined_df.columns)[1],
                yaxis='y1'
            )
        )

        # Add second trace (RHS y-axis)
        fig.add_trace(
            go.Scatter(
                x=combined_df['Date'],
                y=combined_df[list(combined_df.columns)[2]],
                name=list(combined_df.columns)[2],
                yaxis='y2'
            )
        )

        # Update layout to include secondary y-axis
        fig.update_layout(
            title=title,
            xaxis=dict(
                title='Date'
            ),
            yaxis=dict(
                title=fr_unit
            ),
            yaxis2=dict(
                title='No. of vessels',
                overlaying='y',
                side='right'
            ),
            xaxis_rangeslider_visible=True
        )

        fig.show()
    
#     combined_df['Date']=combined_df['Date'].dt.strftime('%d/%m/%Y')
    
    return combined_df

# Function to test causality between availability and a freight rate for multiple laycan and moving average combinations
def find_highest_causality(ma_max, ma_step, laycan_min, laycan_max, laycan_step, max_lag, alpha, start_y, start_m, start_d, region, port, prod, prod_excl, vessel_class, vessel_class_excl, heading_to, heading_to_excl, status, rates, unit):

    # intialise a data frame to store test results
    results_df=pd.DataFrame()

    # create list of MA periods to use
    mas_to_use=[]
    for i in range(0, (ma_max + ma_step), ma_step):

        if i==0:
            mas_to_use.append(1)

        else:
            mas_to_use.append(i)

    mas_to_use=list(set(mas_to_use))
    mas_to_use.sort()
    mas_to_use=[x for x in mas_to_use if x <= ma_max]


    # create list of laycans to use
    laycans_to_use=[]
    for i in range(laycan_min, (laycan_max+1), laycan_step):

        for j in range((i+1), (laycan_max+1)):

            add=(i, j)
            laycans_to_use.append(add)


    # loop through each laycan and each MA period
    for laycan in laycans_to_use:

        for ma_p in mas_to_use:

            # update me
            print(f'Currently loading {ma_p}-day MA data for {laycan[0]} to {laycan[1]} days')

            # query availability & freight price date combination
            data=fr_va_combination(start_y=start_y, start_m=start_m, start_d=start_d, 
                      region=region, port=port, 
                      prod=prod, prod_excl=prod_excl, 
                      vessel_class=vessel_class, vessel_class_excl=vessel_class_excl, 
                      heading_to=heading_to, heading_to_excl=heading_to_excl, 
                      status=status, laycan_min=laycan[0], laycan_max=laycan[1], va_ma_period=ma_p,
                      rates=rates, unit=unit, plot=False, 
                      title=None)

            # test data
            test_result = grangercausalitytests(data[[list(data.columns)[2], list(data.columns)[1]]], max_lag, verbose=False)

            # initialise lists to store combination details and test results
            laycan_list=[]
            p_vals_list=[]
            lags_list=[]
            mas_list=[]

            for lag in range(1, (max_lag + 1), 1):

                laycan_add=f'{laycan[0]} - {laycan[1]} days'
                p_val=test_result[lag][0]['ssr_ftest'][1]

                laycan_list.append(laycan_add)
                p_vals_list.append(p_val)
                lags_list.append(lag)
                mas_list.append(ma_p)

            # convert to data frames
            laycan_df=pd.DataFrame(laycan_list, columns=['laycan'])
            p_vals_df=pd.DataFrame(p_vals_list, columns=['p-value'])
            lags_df=pd.DataFrame(lags_list, columns=['lag'])
            mas_df=pd.DataFrame(mas_list, columns=['ma_period'])

            # combine results
            granger_df=pd.concat([laycan_df, p_vals_df, lags_df, mas_df], axis=1)

            # add to master
            results_df=pd.concat([results_df, granger_df])
            
            
    # sort by statistical significance and reset indices
    results_df.sort_values(by='p-value', ascending=True, inplace=True)
    results_df.reset_index(drop=True, inplace=True)
    
    # apply Bonferonni correction
    corrected_alpha=alpha/len(results_df)
    print(" ")
    print(f'Corrected alpha level based on {len(results_df)} tests: {corrected_alpha}')
    results_df.loc[results_df['p-value']<=corrected_alpha, 'decision'] = 'causality'
    results_df.loc[results_df['p-value']>corrected_alpha, 'decision'] = 'no causality'

    
    return results_df

# Worked example

First, we need to loop through a few combinations of laycan periods and moving averages to find combinations of interest.

In [ ]:
c_test=find_highest_causality(ma_max=21, ma_step=7, 
                              laycan_min=5, laycan_max=25, laycan_step=5, 
                              max_lag=10, alpha=0.05, 
                              start_y=2024, start_m=4, start_d=1, 
                              region=padd3, port=None, 
                              prod=crude_co, prod_excl=None, 
                              vessel_class='oil_aframax_lr2', vessel_class_excl=None, 
                              heading_to=None, heading_to_excl=None, status=None, 
                              rates=['TD25'], unit='cost')


    

Now let's view the results sorted by p-value from most to least significant. 

**Note:** Some results may not be *technically* statistically significant, however, can still exhibit strong predictability when viewed alongside freight rates. The choice of candidates depends on technical suitability as well as suitability in the context of the market being analysed.

In [ ]:
c_test

Filter for a specific lag if desired.

In [ ]:
c_test[(c_test['lag']>=3)]

Now we can query candidate combinations to see how the time series interact with each other.

In [ ]:
tryouts=fr_va_combination(start_y=2024, start_m=4, start_d=1, 
                          region=padd3, port=None, 
                          prod=crude_co, prod_excl=None, 
                          vessel_class='oil_aframax_lr2', vessel_class_excl=None, 
                          heading_to=None, heading_to_excl=None, 
                          status=None, laycan_min=4, laycan_max=8, va_ma_period=14,
                          rates=['TD25'], unit='cost', plot=True, 
                          title='TD25 freight rate vs Aframax availability in the GoM')